# Proteomic Insights into Inflammatory Bowel Disease biomarker discovery in the UK Biobank.


**Authors:**
Francisco Salamanca¹, David Gomez², Daniel Bonilla³

**Affiliations:**
1. MSc Bioinformatics Student, Universidad Nacional de Colombia  
2. MSc Industrial Engineering Student, Universidad Nacional de Colombia 
3. System Engineering Student, Universidad Nacional de Colombia


**Objective:**
The primary objective of this project is to develop a predictive model for inflammatory bowel disease (IBD) relapse and new onset, leveraging longitudinal proteomics data from the UK Biobank. The model aims to capture early signals of disease activity or onset, particularly focusing on proteomic biomarkers trajectories.

**Background:**
IBD is a chronic and relapsing inflammatory disorder that includes Crohn’s disease and ulcerative colitis. Despite advances in treatment, predicting disease progression and relapse remains a major clinical challenge. Multi-omics profiling provides a promising avenue for identifying molecular signatures associated with IBD activity over time.

**Approach:**
- **Data**: UK Biobank data including proteomics (Olink panels), genomics, and clinical data.
- **Participants**: Individuals with multiple time-point measurements for proteomics and relevant metadata.
- **Outcome Variables**: 
    - IBD diagnosis and subtypes (if available),
    - Relapse indicators or clinical events related to disease progression.

**Methods:**
- Preprocessing of datasets and harmonization of participant identifiers.
- Extraction of **relevant** clinical covariates (e.g., medication use, smoking, alcohol, BMI) from the datasets.
- Identification of temporal patterns and trajectory modeling of proteomic profiles.
- Machine learning models (e.g., random forest, survival models, neural networks) adjusted for medical variables known to be associated with IBD.
- Validation using cross-validation and/or independent subsets of the data.

**Expected Results:**
- Identification of omics-based biomarkers predictive of IBD relapse or future diagnosis.
- Insights into the molecular mechanisms underlying disease progression.
- A prototype predictive tool to aid in risk stratification and early clinical intervention.

**Deliverables:**
- A cleaned longitudinal dataset.
- Statistical and machine learning models with performance metrics.
- Visualizations of longitudinal profiles and feature importance.
- Final report and optional manuscript draft for publication.

### Load Packages and Paths



In [ ]:
#install required packages
#!pip install pandas
#!pip install numpy 
#!pip install matplotlib
#!pip install seaborn
#!pip install re

In [2]:
#Load modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
from datetime import datetime

In [5]:
path= "/Users/fjosesala/Documents/IKMB/DATA/UK_BIOBANK_DATA/"

### Load Data and Initial exploration



#### UKB Data

##### Proteomics

In [19]:
#Charge dataframes
#Proteomics
Proteomics_df = pd.read_csv(os.path.join(path, "olink_data.tsv"), sep="\t")
Proteomics_chars_df = pd.read_csv(os.path.join(path, "Proteomics_modified_data/olink_chars_table.tsv"), sep="\t")

#Crear columna con el nombre de las proteinas y #ordenar df
cols = Proteomics_chars_df.columns.tolist()
Proteomics_chars_df.insert(cols.index("ValueType"), "pname",Proteomics_chars_df["Field"].str.split(";").str[0])
Proteomics_chars_df.head()

,Field,FieldID,pname,ValueType,Coding,Category,Participants,Items,Stability,Units,ItemType,Strata,Sexed,Instances,Array,Notes,Link,Path
0,A1BG;Alpha-1B-glycoprotein,183801,A1BG,float,NaN,1838,NaN,NaN,Complete,NaN,Data,Primary,Unisex,0,1,A1BG;Alpha-1B-glycoprotein: a1bg: Protein meas...,http://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=...,Biological samples > Blood assays > Proteomics...
1,AAMDC;Mth938 domain-containing protein,183802,AAMDC,float,NaN,1838,NaN,NaN,Complete,NaN,Data,Primary,Unisex,0,1,AAMDC;Mth938 domain-containing protein: aamdc:...,http://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=...,Biological samples > Blood assays > Proteomics...
2,AARSD1;Alanyl-tRNA editing protein Aarsd1,183803,AARSD1,float,NaN,1838,NaN,NaN,Complete,NaN,Data,Primary,Unisex,3,1,AARSD1;Alanyl-tRNA editing protein Aarsd1: aar...,http://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=...,Biological samples > Blood assays > Proteomics...
3,ABCA2;ATP-binding cassette sub-family A member 2,183804,ABCA2,float,NaN,1838,NaN,NaN,Complete,NaN,Data,Primary,Unisex,0,1,ABCA2;ATP-binding cassette sub-family A member...,http://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=...,Biological samples > Blood assays > Proteomics...
4,ABHD14B;Protein ABHD14B,183805,ABHD14B,float,NaN,1838,NaN,NaN,Complete,NaN,Data,Primary,Unisex,3,1,ABHD14B;Protein ABHD14B: abhd14b: Protein meas...,http://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=...,Biological samples > Blood assays > Proteomics...


In [20]:
#Encuentra columnas duplicadas y eliminalas
Proteomics_df = Proteomics_df.loc[:, ~Proteomics_df.columns.duplicated()]

In [21]:
##Cambiar las columnas de Olink_proteomics, pasarlas de formato: FieldID_instance a formato: Nombre_instancia

# Extraer columnas (excepto 'eid')
original_cols = Proteomics_df.columns.tolist()
data_cols = original_cols[1:]

# Extraer FieldID y instance de los nombres de columna
fids = [int(re.search(r'f_(\d+)_', col).group(1)) if re.search(r'f_(\d+)_', col) else None for col in data_cols]
instances = [re.sub(r'f_\d+_|f_NA_', '', col) for col in data_cols]

# Crear DataFrame auxiliar
col_df = pd.DataFrame({'old': data_cols, 'fids': fids, 'instance': instances})

# Unir con olinkchar para obtener los 'pname'
col_df = col_df.merge(Proteomics_chars_df[['FieldID', 'pname']], left_on='fids', right_on='FieldID', how='left')

# Crear nuevos nombres
col_df['newname'] = col_df['pname'] + '_' + col_df['instance']

# Asignar nuevos nombres de columna
Proteomics_df.columns = ['eid'] + col_df['newname'].tolist()

# Obtener la columna 'eid'
eids = Proteomics_df['eid']

In [ ]:
Proteomics_df.head()

,eid,A1BG_0,AAMDC_0,AARSD1_0,ABCA2_0,ABHD14B_0,ABL1_0,ABO_0,ABRAXAS2_0,ACAA1_0,...,TST_3,TXLNA_3,TXNDC15_3,TXNDC5_3,TXNRD1_3,TYRO3_3,UBAC1_3,UMOD_3,USO1_3,UXS1_3
0,1000428,-0.4161,-0.34175,0.45155,0.0372,0.4493,0.9716,1.6605,0.2928,0.6318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000733,-0.2114,0.08865,0.19445,-0.3721,-0.2925,-0.2254,-1.6076,0.0739,-0.2268,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000786,0.3317,0.52155,0.36340,0.0541,0.3679,0.5792,-0.5366,0.3326,0.3272,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000952,0.6352,0.20735,1.08470,0.1064,0.9497,1.8300,0.2250,1.8695,1.2220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1001279,0.0081,-0.62070,-0.02600,0.5784,0.2625,0.5433,-1.7003,0.3429,1.4269,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Convert column names to strings
Proteomics_df.columns = Proteomics_df.columns.astype(str)

# Hay proteinas que no estan en las 4 sets longitudinales, por lo que se tiene que decidir o tener en cuenta!!!
# Ejemplo:
filtered_columns = [col for col in Proteomics_df.columns if col.startswith('CD6')]
print(filtered_columns)

['CD6_0', 'CD63_0', 'CD69_0', 'CD6_2', 'CD63_2', 'CD69_2', 'CD6_3', 'CD63_3', 'CD69_3']


##### Phenotypes

In [56]:
#Charge the dataframes
phenotypes_df = pd.read_csv(os.path.join(path,"phenotype_data.tsv"), sep="\t")
phenotypes_df.head()

,eid,f_1239_0_0,f_1239_1_0,f_1239_2_0,f_1239_3_0,f_1249_0_0,f_1249_1_0,f_1249_2_0,f_1249_3_0,f_1259_0_0,...,f_22037_2_0,f_22037_3_0,f_22038_0_0,f_22038_1_0,f_22038_2_0,f_22038_3_0,f_22039_0_0,f_22039_1_0,f_22039_2_0,f_22039_3_0
0,1000012,0.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000021,0.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,...,NaN,NaN,4320.0,NaN,NaN,NaN,320.0,NaN,NaN,NaN
2,1000035,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
3,1000044,0.0,0.0,NaN,NaN,2.0,1.0,NaN,NaN,1.0,...,NaN,NaN,0.0,240.0,NaN,NaN,0.0,960.0,NaN,NaN
4,1000053,0.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,0.0,...,NaN,NaN,240.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [57]:
##Phenotypes DF ####
##¿Cuales fenotipos estan mas associados a la condicion de tener ibd?
#Se sabe por bibliografia que: smoking y alcohol

# Seleccionar columnas cuyos nombres contienen _20116_ o _20117_
phenodata_esential_df = phenotypes_df.filter(regex=r'eid|_20116_|_20117_', axis=1)

# Crear un diccionario para renombrar columnas
new_columns = {
    col: col.replace('f_20116', 'Smoking').replace('f_20117', 'Alcohol').removesuffix('_0')
    for col in phenodata_esential_df.columns
}

# Renombrar las columnas
phenodata_esential_df.rename(columns=new_columns, inplace=True)

# Mostrar el DataFrame resultante
phenodata_esential_df.head()



/var/folders/bf/fdxk2h0901v934ldwnk_10x00000gn/T/ipykernel_5280/4211983069.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  phenodata_esential_df.rename(columns=new_columns, inplace=True)


,eid,Smoking_0,Smoking_1,Smoking_2,Smoking_3,Alcohol_0,Alcohol_1,Alcohol_2,Alcohol_3
0,1000012,0.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN
1,1000021,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN
2,1000035,2.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN
3,1000044,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN
4,1000053,0.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN


In [58]:
# Reemplazar -3 por np.nan en columnas numéricas, excepto 'eid'
cols_to_replace = phenodata_esential_df.select_dtypes(include='number').columns.drop('eid', errors='ignore')
phenodata_esential_df[cols_to_replace] = phenodata_esential_df[cols_to_replace].replace(-3, np.nan)
phenodata_esential_df.head()

/var/folders/bf/fdxk2h0901v934ldwnk_10x00000gn/T/ipykernel_5280/2126187986.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  phenodata_esential_df[cols_to_replace] = phenodata_esential_df[cols_to_replace].replace(-3, np.nan)


,eid,Smoking_0,Smoking_1,Smoking_2,Smoking_3,Alcohol_0,Alcohol_1,Alcohol_2,Alcohol_3
0,1000012,0.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN
1,1000021,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN
2,1000035,2.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN
3,1000044,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN
4,1000053,0.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN


##### Etnicity
*This table contains Etnicity info of the different participants according to the genetics. is coded in a PCA base*

In [62]:
#Charge dataframes
etnicidad_df = pd.read_csv(os.path.join(path,"genomics_data.tsv"), sep="\t")
etnicidad_df.head()

,eid,f_22000_0_0,f_22001_0_0,f_22003_0_0,f_22004_0_0,f_22005_0_0,f_22006_0_0,f_22009_0_1,f_22009_0_2,f_22009_0_3,...,f_22024_0_0,f_22025_0_0,f_22026_0_0,f_22027_0_0,f_22028_0_0,f_22029_0_0,f_22030_0_0,f_22007_0_0,f_22008_0_0,f_22182_0_0
0,1000012,10.0,0.0,0.186874,0.188240,0.008535,1.0,-12.4522,2.97502,-1.526670,...,44.7400,99.059,0.97000,NaN,1.0,1.0,1.0,SMP4_0010894A,G03,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,..."
1,1000021,-7.0,1.0,0.190092,0.191694,0.001877,1.0,-13.8955,6.01974,-2.011150,...,93.3443,99.477,0.95280,NaN,1.0,1.0,1.0,SMP4_0008357,B08,"0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
2,1000035,-2.0,0.0,0.190206,0.191802,0.001883,1.0,-11.7626,4.23129,-3.091340,...,30.1373,99.595,0.96018,NaN,1.0,1.0,1.0,SMP4_0008467,C11,"1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,..."
3,1000044,57.0,1.0,0.190249,0.191653,0.003119,1.0,-11.2151,4.37638,0.766213,...,29.3400,99.580,0.98000,NaN,1.0,1.0,1.0,SMP4_0011008A,A03,"0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,..."
4,1000053,69.0,0.0,0.190025,0.191251,0.002626,1.0,-10.4144,2.19996,0.545220,...,22.1800,99.572,0.97000,NaN,1.0,1.0,1.0,SMP4_0013058A,H12,"0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."


In [63]:
#Filtrar por el id "Genetic principal components (22009)", el campo el cual me indica etnicidad 

etnicidad_df.columns = etnicidad_df.columns.str.replace("f_22009_0", "PC", regex=True)
etnicidad_df.head()

,eid,f_22000_0_0,f_22001_0_0,f_22003_0_0,f_22004_0_0,f_22005_0_0,f_22006_0_0,PC_1,PC_2,PC_3,...,f_22024_0_0,f_22025_0_0,f_22026_0_0,f_22027_0_0,f_22028_0_0,f_22029_0_0,f_22030_0_0,f_22007_0_0,f_22008_0_0,f_22182_0_0
0,1000012,10.0,0.0,0.186874,0.188240,0.008535,1.0,-12.4522,2.97502,-1.526670,...,44.7400,99.059,0.97000,NaN,1.0,1.0,1.0,SMP4_0010894A,G03,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,..."
1,1000021,-7.0,1.0,0.190092,0.191694,0.001877,1.0,-13.8955,6.01974,-2.011150,...,93.3443,99.477,0.95280,NaN,1.0,1.0,1.0,SMP4_0008357,B08,"0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
2,1000035,-2.0,0.0,0.190206,0.191802,0.001883,1.0,-11.7626,4.23129,-3.091340,...,30.1373,99.595,0.96018,NaN,1.0,1.0,1.0,SMP4_0008467,C11,"1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,..."
3,1000044,57.0,1.0,0.190249,0.191653,0.003119,1.0,-11.2151,4.37638,0.766213,...,29.3400,99.580,0.98000,NaN,1.0,1.0,1.0,SMP4_0011008A,A03,"0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,..."
4,1000053,69.0,0.0,0.190025,0.191251,0.002626,1.0,-10.4144,2.19996,0.545220,...,22.1800,99.572,0.97000,NaN,1.0,1.0,1.0,SMP4_0013058A,H12,"0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."


##### Physical Measures
*This table contains clinical info of the different participants, such as Weight, Height, Artritis, Pregnancy etc*

In [ ]:
#Charge dataframes
physical_measures_df = pd.read_csv(os.path.join(path, "Physical_measures_data.tsv"), sep="\t")
physical_measures_df.head()

In [ ]:
# 1. Seleccionar columnas relevantes de physical_measures_df
columns_to_keep = ['eid'] + [col for col in physical_measures_df.columns if re.search(r'_48_|_49_|_23104_|_23100_|_23101_|_23102_', col)]
phys_esential = physical_measures_df[columns_to_keep].copy()


# 2. Calcular WHR para cada visita (0 a 3), Crear una nueva columna 'WHR' para cada punto, donde sea la relacion entre la cintura y la cadera
for i in range(4):
    phys_esential[f'WHR_{i}'] = phys_esential[f'f_48_{i}_0'] / phys_esential[f'f_49_{i}_0']

# 3. Renombrar columnas
phys_esential.columns = (
    phys_esential.columns
    .str.replace('f_48', 'Waist', regex=True)
    .str.replace('f_49', 'Hip', regex=True)
    .str.replace('f_23104', 'BMI', regex=True)
    .str.replace('f_23100', 'wFatMass', regex=True)
    .str.replace('f_23101', 'wFatFreeMass', regex=True)
    .str.replace('f_23102', 'wWaterMass', regex=True)
    .str.replace('_0$', '', regex=True)
    .str.replace(r'WHR$', 'WHR_0', regex=True)
)

phys_esential.head()

,eid,Waist_0,Waist_1,Waist_2,Waist_3,Hip_0,Hip_1,Hip_2,Hip_3,wFatMass_0,...,wWaterMass_2,wWaterMass_3,BMI_0,BMI_1,BMI_2,BMI_3,WHR_0,WHR_1,WHR_2,WHR_3
0,1000012,77.0,NaN,NaN,NaN,92.0,NaN,NaN,NaN,19.8,...,NaN,NaN,24.0,NaN,NaN,NaN,0.836957,NaN,NaN,NaN
1,1000021,102.0,NaN,NaN,NaN,109.0,NaN,NaN,NaN,20.0,...,NaN,NaN,28.7,NaN,NaN,NaN,0.935780,NaN,NaN,NaN
2,1000035,83.0,NaN,NaN,NaN,95.0,NaN,NaN,NaN,22.0,...,NaN,NaN,25.0,NaN,NaN,NaN,0.873684,NaN,NaN,NaN
3,1000044,120.0,114.0,NaN,NaN,117.0,111.0,NaN,NaN,43.0,...,NaN,NaN,38.8,NaN,NaN,NaN,1.025641,1.027027,NaN,NaN
4,1000053,78.0,NaN,NaN,NaN,102.0,NaN,NaN,NaN,24.3,...,NaN,NaN,22.5,NaN,NaN,NaN,0.764706,NaN,NaN,NaN


##### First Ocurrences 
*This table says info about the differents diseases along the DB*

In [6]:
#Load dataframes
first_occurences_df = pd.read_csv(os.path.join(path, "FirstOccurrences_data.tsv"), sep="\t")
first_occurences_chars_df = pd.read_csv(os.path.join(path, "FirstOccurrences_chars.tsv"), sep="\t")

/var/folders/bf/fdxk2h0901v934ldwnk_10x00000gn/T/ipykernel_5280/384883312.py:2: DtypeWarning: Columns (1,3,4,5,7,8,10,11,12,13,14,15,16,17,20,21,22,23,24,25,26,27,29,30,31,32,33,34,35,36,37,39,40,41,42,43,44,45,46,47,48,49,50,52,53,54,55,56,58,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,81,82,83,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,101,103,104,105,106,107,110,111,112,113,114,115,118,119,120,121,122,124,125,127,128,129,130,131,132,133,136,139,140,141,143,145,146,147,148,149,151,152,154,155,156,158,159,160,161,162,163,164,165,167,168,169,170,171,172,174,175,176,177,178,180,181,182,184,185,186,188,189,190,191,192,194,196,198,199,200,202,203,204,205,207,208,209,210,211,212,213,214,216,217,218,219,220,223,224,225,227,228,229,230,231,232,233,234,235,236,239,241,242,243,244,246,248,249,250,252,254,256,257,258,259,260,261,262,263,264,265,266,267,268,269,272,273,274,275,277,278,279,280,281,282,283,284,285,286,288,289,290,291,292,293,294,297,301,304,305,306,307,308,309,310,3

##### PopChar


In [ ]:
#Load dataframes
popchar_df = pd.read_csv(os.path.join(path, "popchar_data.tsv"), sep="\t")
popchar_df.head()

In [25]:
# 1. Seleccionar columnas relevantes de popchars_df
columns_to_keep = ['eid'] + [col for col in popchar_df.columns if re.search(r'_31_|_34_|_52_', col)]
popchars_esential = popchar_df[columns_to_keep].copy()

# 2. Renombrar columnas
popchars_esential.columns = (
    popchars_esential.columns
    .str.replace('f_31', 'Sex', regex=True)
    .str.replace('f_34', 'YearofBirth', regex=True)
    .str.replace('f_52', 'MonthofBirth', regex=True)
    .str.replace('_0$', '', regex=True)
)

popchars_esential.head()

,eid,Sex_0,YearofBirth_0,MonthofBirth_0
0,1000012,0,1947,2
1,1000021,1,1952,9
2,1000035,0,1964,10
3,1000044,1,1959,7
4,1000053,0,1946,4


##### Recruitment

In [17]:
#Load dataframes
recruitment_df = pd.read_csv(os.path.join(path, "recruitment_data.tsv"), sep="\t")
recruitment_df.head()

,eid,f_53_0_0,f_53_1_0,f_53_2_0,f_53_3_0,f_54_0_0,f_54_1_0,f_54_2_0,f_54_3_0,f_55_0_0,f_55_1_0,f_55_2_0,f_55_3_0,f_20118_0_0,f_21003_0_0,f_21003_1_0,f_21003_2_0,f_21003_3_0
0,1000012,2009-05-16,NaN,NaN,NaN,11016,NaN,NaN,NaN,5,NaN,NaN,NaN,5.0,62,NaN,NaN,NaN
1,1000021,2009-07-27,NaN,NaN,NaN,11016,NaN,NaN,NaN,7,NaN,NaN,NaN,5.0,56,NaN,NaN,NaN
2,1000035,2008-05-07,NaN,NaN,NaN,11009,NaN,NaN,NaN,5,NaN,NaN,NaN,5.0,43,NaN,NaN,NaN
3,1000044,2009-04-22,2012-11-15,NaN,NaN,11016,11024.0,NaN,NaN,4,11.0,NaN,NaN,5.0,49,53.0,NaN,NaN
4,1000053,2009-09-17,NaN,NaN,NaN,11011,NaN,NaN,NaN,9,NaN,NaN,NaN,6.0,63,NaN,NaN,NaN


##### TouchScreen

In [ ]:
#Load dataframes
touchscreen_df = pd.read_csv(os.path.join(path, "touchscreen_data.tsv"), sep="\t")
touchscreen_chars_df = pd.read_csv(os.path.join(path, "touchscreen_chars.tsv"), sep="\t")


### Preprocessing (Raw to tidy)

#### UKB

### Covariable Selection for IBD

### Final Dataset Assembly

In [28]:
## Coger todas las fechas de FIRSTOCCURRENCES

dates_raw = []

# Recolectar todas las fechas únicas de cada columna a partir de la columna 2
for col in first_occurences_df.columns[1:]:
    unique_vals = first_occurences_df[col].dropna().unique()
    dates_raw.extend([str(val) for val in unique_vals])

# Quitar duplicados
dates_raw = list(set(dates_raw))

# Eliminar fechas específicas por posición (como hiciste con -c(...) en R)
# Asegúrate de que esos índices existan
bad_indices = [19006, 19007, 19008, 19009, 19050]  # Ajustado a base-0
dates_raw0 = [date for i, date in enumerate(dates_raw) if i not in bad_indices]

# Intentar convertir a fechas
dates = []
failed_dates = []

for date_str in dates_raw0:
    try:
        parsed = pd.to_datetime(date_str, errors='raise')
        dates.append(parsed)
    except:
        failed_dates.append(date_str)

# Reintentar con las fallidas (a veces el segundo intento funciona con otro formato)
for date_str in failed_dates:
    try:
        parsed = pd.to_datetime(date_str, errors='raise')
        dates.append(parsed)
    except:
        pass  # Puedes guardar las que siguen fallando si quieres analizarlas

# Ordenar y eliminar duplicados
dates = sorted(set(dates))

print(dates)

[Timestamp('1937-11-01 00:00:00'), Timestamp('1938-07-01 00:00:00'), Timestamp('1938-08-01 00:00:00'), Timestamp('1938-09-01 00:00:00'), Timestamp('1938-10-01 00:00:00'), Timestamp('1938-11-01 00:00:00'), Timestamp('1939-02-01 00:00:00'), Timestamp('1939-03-01 00:00:00'), Timestamp('1939-04-01 00:00:00'), Timestamp('1939-05-01 00:00:00'), Timestamp('1939-07-01 00:00:00'), Timestamp('1939-08-01 00:00:00'), Timestamp('1939-09-01 00:00:00'), Timestamp('1939-10-01 00:00:00'), Timestamp('1939-11-01 00:00:00'), Timestamp('1940-01-01 00:00:00'), Timestamp('1940-02-01 00:00:00'), Timestamp('1940-03-01 00:00:00'), Timestamp('1940-04-01 00:00:00'), Timestamp('1940-05-01 00:00:00'), Timestamp('1940-07-01 00:00:00'), Timestamp('1940-08-01 00:00:00'), Timestamp('1940-09-01 00:00:00'), Timestamp('1940-10-01 00:00:00'), Timestamp('1940-11-01 00:00:00'), Timestamp('1941-01-01 00:00:00'), Timestamp('1941-01-09 00:00:00'), Timestamp('1941-02-01 00:00:00'), Timestamp('1941-03-01 00:00:00'), Timestamp('19

In [29]:
#Filtrar todas las enfermedades gastrointestinales

# 1. Obtener el último término de cada jerarquía en la columna 'Path'

disease_umbrellas = (
    first_occurences_chars_df['Path']
    .dropna()
    .unique()
)

# Aplicar transformación a cada string: dividir por '>', invertir, tomar el primero y limpiar espacios
disease_umbrellas = [
    path.split('>')[-1].strip() for path in disease_umbrellas
]

# 2. Filtrar por 'Digestive system disorders' y 'Date'
digestive_disorders = first_occurences_chars_df[
    first_occurences_chars_df['Path'].str.contains('Digestive system disorders', na=False) &
    (first_occurences_chars_df['ValueType'] == 'Date')
]


print(digestive_disorders)




                                                 Field  FieldID ValueType  \
622  Date K01 first reported (embedded and impacted...   131554      Date   
624            Date K02 first reported (dental caries)   131556      Date   
626  Date K04 first reported (diseases of pulp and ...   131560      Date   
628  Date K05 first reported (gingivitis and period...   131562      Date   
630  Date K06 first reported (other disorders of gi...   131564      Date   
632  Date K07 first reported (dentofacial anomalies...   131566      Date   
634  Date K08 first reported (other disorders of te...   131568      Date   
636  Date K09 first reported (cysts of oral region,...   131570      Date   
638   Date K10 first reported (other diseases of jaws)   131572      Date   
640  Date K11 first reported (diseases of salivary ...   131574      Date   
642  Date K12 first reported (stomatitis and relate...   131576      Date   
644  Date K13 first reported (other diseases of lip...   131578      Date   

In [30]:
# Filtrar first_occurences_df por los ids de enfermedades con IBD: UC y CD

# Filtrar campos que contienen K52, K50 o K51 en la columna 'Field', y que sean del tipo 'Date'
ibd_fields = first_occurences_chars_df[
    first_occurences_chars_df['Field'].str.contains('K52|K50|K51', na=False) &
    (first_occurences_chars_df['ValueType'] == 'Date')
]

print(ibd_fields)

                                                 Field  FieldID ValueType  \
684  Date K50 first reported (crohn's disease [regi...   131626      Date   
686       Date K51 first reported (ulcerative colitis)   131628      Date   
688  Date K52 first reported (other non-infective g...   131630      Date   

     Coding  Category  Participants  Items Stability  Units ItemType   Strata  \
684     819      2411          3502   3502   Ongoing    NaN     Data  Derived   
686     819      2411          6698   6698   Ongoing    NaN     Data  Derived   
688     819      2411         31145  31145   Ongoing    NaN     Data  Derived   

      Sexed  Instances  Array  \
684  Unisex          1      1   
686  Unisex          1      1   
688  Unisex          1      1   

                                                 Notes  \
684  Date of the first occurrence of any code mappe...   
686  Date of the first occurrence of any code mappe...   
688  Date of the first occurrence of any code mappe...   



In [34]:
#Filtrar por individuos que tienen olink

# Filtrar recruitmentdata_raw para quedarse con los individuos con datos Olink
olink_recruits = recruitment_df[recruitment_df['eid'].isin(eids)]

# Filtrar first ocurrences también por los mismos IDs
olink_first_ocurrences = first_occurences_df[first_occurences_df['eid'].isin(eids)]


# de ibd_fields es un DataFrame con una columna 'FieldID'
field_ids = ibd_fields['FieldID'].astype(str).tolist()

# Crear patrón regex: '131626|131628|131630|...'
pattern = '|'.join(field_ids)

# Filtrar columnas que contienen alguna coincidencia con los IDs + 'eid'
columns_to_keep = ['eid'] + [col for col in olink_first_ocurrences.columns if re.search(pattern, col)]

# Seleccionar columnas
fosid = fosi[columns_to_keep].copy()

# Renombrar columnas: quitar 'f_' y '_0', luego reemplazar IDs por etiquetas
fosid.columns = (
    fosid.columns
    .str.replace('f_', '', regex=False)
    .str.replace('_0', '', regex=False)
    .str.replace('131626', 'CD', regex=False)
    .str.replace('131628', 'UC', regex=False)
    .str.replace('131630', 'IBD', regex=False)
)


fosid.head()


,eid,CD,UC,IBD
11,5120517,NaN,NaN,NaN
31,2068490,NaN,NaN,NaN
42,1169048,NaN,NaN,NaN
49,1040674,NaN,NaN,NaN
50,4191600,NaN,NaN,NaN


In [ ]:
# Parte 1: contar valores no nulos por columna
non_na_counts = fosid.notna().sum()


# Parte 2: obtener los IDs únicos de pacientes enfermos
sick = pd.concat([
    fosid.loc[fosid['CD'].notna(), 'eid'],
    fosid.loc[fosid['UC'].notna(), 'eid'],
    fosid.loc[fosid['IBD'].notna(), 'eid']
]).unique()


In [ ]:
#Filtrar las columnas que están relacionadas con trastornos digestivos

# Crear un patrón regex a partir de los FieldIDs 
pattern = '|'.join(map(str, digestive_disorders['FieldID']))

# Seleccionar columnas: 'eid' + aquellas que coincidan con el patrón
ddsid = fosi[['eid'] + [col for col in fosi.columns if re.search(pattern, col)]]

ddsid


In [41]:
# Filtrar filas donde todas las columnas excepto 'eid' son NA

digestive_cols = ddsid.columns.drop('eid')
mask = ddsid[digestive_cols].isna().sum(axis=1) == len(digestive_cols)
popcontrols = ddsid[mask]

# Obtener los Ids de los controles poblacionales (Sin enfermedades digestivas) )
popcontrolsids = popcontrols['eid'].values


In [ ]:

# --- Paso 1: Seleccionar columnas específicas de olink_recruits ---
timestamps = olink_recruits.loc[:, [
    'eid',
    'f_53_0_0', 'f_53_1_0', 'f_53_2_0', 'f_53_3_0',
    'f_21003_0_0', 'f_21003_1_0', 'f_21003_2_0', 'f_21003_3_0',
    'f_54_0_0'
]].copy()

# --- Paso 2: Renombrar columnas ---
timestamps.columns = [
    'eid', 'TM1', 'TM2', 'TM3', 'TM4',
    'Age1', 'Age2', 'Age3', 'Age4',
    'AssessmentCentre'
]

# --- Paso 3: Hacer merge con fosid por 'eid' ---
infodata = pd.merge(fosid, timestamps, on='eid', how='left')

# --- Paso 4: Convertir columnas de fechas a datetime ---
date_cols = ['CD', 'UC', 'IBD', 'TM1', 'TM2', 'TM3', 'TM4']
for col in date_cols:
    infodata[col] = pd.to_datetime(infodata[col], errors='coerce')

# --- Paso 5: Calcular variables tiempo relativo a diagnóstico ---
for disease in ['CD', 'UC', 'IBD']:
    for timepoint in ['TM1', 'TM2', 'TM3', 'TM4']:
        infodata[f'{disease}2{timepoint}'] = (infodata[disease] - infodata[timepoint]).dt.days

# --- Paso 6: Pivotear las columnas CD, UC, IBD a formato largo ---
infodata_long = infodata.melt(
    id_vars=[col for col in infodata.columns if col not in ['CD', 'UC', 'IBD']],
    value_vars=['CD', 'UC', 'IBD'],
    var_name='Disease',
    value_name='DiagnosedAt'
)

# --- Paso 7: Calcular diferencias DxTM1 a DxTM4 (días entre diagnóstico y visitas TM) ---
for i in range(1, 5):
    infodata_long[f'DxTM{i}'] = (pd.to_datetime(infodata_long['DiagnosedAt'], errors='coerce') -
                                pd.to_datetime(infodata_long[f'TM{i}'], errors='coerce')).dt.days

# --- Paso 8: Crear categorías Pre/Post diagnóstico ---
for i in range(1, 5):
    infodata_long[f'TCategory{i}'] = np.where(
        infodata_long[f'DxTM{i}'] < 0,
        'Pre-diagnosis',
        'Post-diagnosis'
    )

# --- Paso 9: Pivotear para reorganizar variables TM, Age, DxTM, TCategory con número de visita ---

# Primero, hacemos melt dejando columnas fijas
id_vars = ['eid', 'Disease', 'DiagnosedAt', 'AssessmentCentre'] + [col for col in infodata_long.columns if not any(s in col for s in ['TM', 'Age', 'DxTM', 'TCategory'])]
melt_vars = [col for col in infodata_long.columns if any(s in col for s in ['TM', 'Age', 'DxTM', 'TCategory'])]

# Pero para simplicidad, hacemos melt explícito con esas columnas:
cols_to_melt = ['TM1', 'TM2', 'TM3', 'TM4',
                'Age1', 'Age2', 'Age3', 'Age4',
                'DxTM1', 'DxTM2', 'DxTM3', 'DxTM4',
                'TCategory1', 'TCategory2', 'TCategory3', 'TCategory4']

df_melt = infodata_long.melt(
    id_vars=['eid', 'Disease', 'DiagnosedAt', 'AssessmentCentre'],
    value_vars=cols_to_melt,
    var_name='VariableTime',
    value_name='Value'
)

# Extraer nombre base y número de visita
df_melt[['Variable', 'Times']] = df_melt['VariableTime'].str.extract(r'([A-Za-z]+)(\d+)')

# Pivot para expandir variables a columnas
infodata1 = df_melt.pivot_table(
    index=['eid', 'Disease', 'DiagnosedAt', 'AssessmentCentre', 'Times'],
    columns='Variable',
    values='Value',
    aggfunc='first'
).reset_index()

# Reordenar columnas si se desea
cols_order = ['eid', 'Disease', 'DiagnosedAt', 'AssessmentCentre', 'Times', 'Age', 'TM', 'DxTM', 'TCategory']
infodata1 = infodata1[cols_order]



In [ ]:
# Filtrar filas con eid en 'sick' y al menos un tiempo relativo a diagnóstico positivo (>0)
mask_sick = infodata0['eid'].isin(sick)
time_cols = ['CD2T1','CD2T2','CD2T3','CD2T4','UC2T1','UC2T2','UC2T3','UC2T4','IBD2T1','IBD2T2','IBD2T3','IBD2T4']

# Crear una máscara que chequea si alguna de las columnas time_cols es > 0
mask_time_positive = infodata0[time_cols].gt(0).any(axis=1)

sickdb = infodata0.loc[mask_sick & mask_time_positive]

# Filtrar sick + controles poblacionales
combined_array = np.concatenate((sick, popcontrolsids))

mask_sickpopcon = infodata0['eid'].isin(combined_array)
sickpopcon = infodata0.loc[mask_sickpopcon]
sickpopcon = infodata0.loc[mask_sickpopcon]

# Solo controles poblacionales
mask_popcon = infodata0['eid'].isin(popcontrolsids)
pocon = infodata0.loc[mask_popcon]

# Filtrar filas donde CD no es NA (no nulo)
sickpopcon_cd = sickpopcon.loc[sickpopcon['CD'].notna()]
infodata0_cd = infodata0.loc[infodata0['CD'].notna()]
cds = infodata0.loc[infodata0['CD'].notna()]

In [51]:
# Filtrar subset de personas con diagnóstico exclusivo

cds_only = sickpopcon.loc[
    sickpopcon['CD'].notna() &
    sickpopcon['UC'].isna() &
    sickpopcon['IBD'].isna()
]

uc_only = sickpopcon.loc[
    sickpopcon['UC'].notna() &
    sickpopcon['CD'].isna() &
    sickpopcon['IBD'].isna()
]

ibd_only = sickpopcon.loc[
    sickpopcon['IBD'].notna() &
    sickpopcon['CD'].isna() &
    sickpopcon['UC'].isna()
]

uc_ibd_only = sickpopcon.loc[
    sickpopcon['UC'].notna() &
    sickpopcon['IBD'].notna() &
    sickpopcon['CD'].isna()
]

cd_ibd_only = sickpopcon.loc[
    sickpopcon['CD'].notna() &
    sickpopcon['IBD'].notna() &
    sickpopcon['UC'].isna()
]

# Filtrar cds_only donde TM2, TM3 y TM4 sean NA
cds_only_missing_TM234 = cds_only.loc[
    cds_only['TM2'].isna() &
    cds_only['TM3'].isna() &
    cds_only['TM4'].isna()
]

# Combinar con controles poblacionales (pocon)
cds_popc = pd.concat([cds_only, pocon], ignore_index=True)
uc_popc = pd.concat([uc_only, pocon], ignore_index=True)
ibd_popc = pd.concat([ibd_only, pocon], ignore_index=True)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Supongamos que `dat` es tu DataFrame equivalente a cds_only

# Por ejemplo, plotear edad en función del tiempo relativo al diagnóstico (YearsRelativeToDx),
# con puntos separados por eid (pacientes) o Disease si hay más

plt.figure(figsize=(10,6))
sns.scatterplot(
    data=cds_only,
    x='YearsRelativeToDx',  # o la variable de tiempo que tengas
    y='Age',                # o variable que quieras visualizar
    hue='eid',              # colorear por paciente, si hay muchos puedes omitir
    s=30                    # tamaño de puntos, ajusta a psize=1.5 como sea necesario
)

plt.title('Diagnoses timeline CDs Only')
plt.xlabel('Years Relative to Diagnosis')
plt.ylabel('Age')

plt.legend([],[], frameon=False)  # oculta leyenda si muchos ids
plt.show()


In [54]:

# Crear lista de eids con solo enfermedad CD, UC o IBD
disease_only_eids = pd.concat([cds_only['eid'], uc_only['eid'], ibd_only['eid']]).unique()

# Combinar los grupos ibd_only, cds_only, uc_only y controles poblacionales (pocon)
disease_popc = pd.concat([ibd_only, cds_only, uc_only, pocon], ignore_index=True)

# Seleccionar columnas específicas en disease_popc (similar a select)
cols_of_interest = ['eid', 'CD', 'UC', 'IBD', 'TM1', 'Age1', 'AssessmentCentre', 'CD2T1', 'UC2T1', 'IBD2T1']
disease_popc_0 = disease_popc.loc[:, cols_of_interest]

#Filtrar solo con los pacientes
donly = infodata1.loc[infodata1['eid'].isin(disease_only_eids)]

#Tablas de conteos 
# Conteo categorías en la visita Times=2 por enfermedad
print(donly.loc[(donly['Disease'] == 'CD') & (donly['Times'] == '2'), 'TCategory'].value_counts())
print(donly.loc[(donly['Disease'] == 'UC') & (donly['Times'] == '2'), 'TCategory'].value_counts())
print(donly.loc[(donly['Disease'] == 'IBD') & (donly['Times'] == '2'), 'TCategory'].value_counts())

#Identificar IDs con post-diagnóstico en la primera visita y filtrar pre-diagnóstico en otras visitas
t1post = donly.loc[
    (donly['Disease'] == 'IBD') & (donly['Times'] == '1') & (donly['TCategory'] == 'Post-diagnosis'),
    'eid'
].unique()

#Contar filas donde para esos eid, en otras visitas no 1, la categoría sea Pre-diagnosis
count_pre_other_times = donly.loc[
    (donly['eid'].isin(t1post)) &
    (donly['Disease'] == 'IBD') &
    (donly['Times'] != '1') &
    (donly['TCategory'] == 'Pre-diagnosis')
].shape[0]

#Filtrar ibd_popc con IBD NA
ibd_na = ibd_popc.loc[ibd_popc['IBD'].isna()]



TCategory
Post-diagnosis    185
Pre-diagnosis       5
Name: count, dtype: int64
TCategory
Post-diagnosis    316
Pre-diagnosis      11
Name: count, dtype: int64
TCategory
Post-diagnosis    3029
Pre-diagnosis       80
Name: count, dtype: int64


In [65]:

#6. Crear dataframe ibd con nuevas variables y joins
# Crear columna Disease con prioridad CD > UC > IBD > Control
def classify_disease(row):
    if pd.notna(row['CD']):
        return 'CD'
    elif pd.notna(row['UC']):
        return 'UC'
    elif pd.notna(row['IBD']):
        return 'IBD'
    else:
        return 'Control'

ibd = disease_popc_0.copy()
ibd['Disease'] = ibd.apply(classify_disease, axis=1)

# Crear columna Time_Category según IBD2T1
ibd['Time_Category'] = np.where(ibd['IBD2T1'] < 0, 'Pre-diagnosis', 'Post-diagnosis')
ibd.loc[ibd['Time_Category'].isna(), 'Time_Category'] = 'Control'

# Convertir Time_Category a categoría ordenada según valores únicos ordenados
unique_levels = sorted(ibd['Time_Category'].dropna().unique())
ibd['Time_Category'] = pd.Categorical(ibd['Time_Category'], categories=unique_levels, ordered=True)

# Hacer left joins (merge) por eid con otros dataframes

# Join con phenodata_esential_df
ibd = ibd.merge(phenodata_esential_df, on='eid', how='left')

# join with PCAS form genomics
pc_columns = ['eid'] + [f'PC_{i}' for i in range(1,6)]
ibd = ibd.merge(etnicidad_df.loc[:, pc_columns], on='eid', how='left')

# Join con olinkdata
ibd = ibd.merge(Proteomics_df, on='eid', how='left')




In [71]:
# Convertimos los nombres de columnas a string
ibd.columns = ibd.columns.astype(str)

# Seleccionar primeras 12 columnas y columnas con PC_ y que terminen en '_0', excepto algunas columnas excluidas
cols = list(ibd.columns[:12]) # primeras 12 column

# Columnas con 'PC_'
cols += [col for col in ibd.columns if 'PC_' in col]

# Columnas que terminan en '_0', excluyendo algunas específicas
cols_0 = [col for col in ibd.columns if col.endswith('_0')]
exclude_cols = ['WHR_0', 'BMI_0', 'YearOfBirth_0', 'MonthOfBirth_0']
cols_0_filtered = [col for col in cols_0 if col not in exclude_cols]

# Concatenar todas las columnas finales
cols += cols_0_filtered

# Subset final
ibd0 = ibd[cols].copy()

# Asegurar que Time_Category tenga orden correcto
ibd0['Time_Category'] = pd.Categorical(
    ibd0['Time_Category'],
    categories=['Control', 'Pre-diagnosis', 'Post-diagnosis'],
    ordered=True
)

#Guardar el dataframe final
ibd0.to_csv(os.path.join(path, "ibd_final_data.tsv"), index=False)


In [ ]:
#Charge dataframes

ibd0 = pd.read_csv(os.path.join(path, "ibd_final_data.tsv"), sep="\t")

print
ibd0

/var/folders/bf/fdxk2h0901v934ldwnk_10x00000gn/T/ipykernel_5280/1435639051.py:3: DtypeWarning: Columns (1,2,3,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  ibd33 = pd.read_csv(os.path.join(path, "ibd0.tsv"), sep="\t")


,eid,CD,UC,IBD,Disease,TM1,season,Age1,AssessmentCentre,CD2T1,...,ZBTB16,ZBTB17,ZCCHC8,ZFYVE19,ZHX2,ZNF174,ZNF75D,ZNF830,ZNRD2,ZNRF4
0,2227199,NaN,NaN,1993-04-07,IBD,2007-12-07,Winter,57,11003,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5822919,NaN,NaN,2021-10-26,IBD,2008-05-19,Spring,51,11008,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3122877,NaN,NaN,2022-05-13,IBD,2009-05-27,Spring,63,11011,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5296207,NaN,NaN,2002-05-30,IBD,2009-03-23,Spring,42,11007,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1847013,NaN,NaN,1999-08-24,IBD,2008-08-01,Summer,65,11007,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Models

#### Model: Logistic Regression per Protein - Biomarker detector

#### MetaAnalysis Between Cohorts

#### Model: Survival analysis (Only UKB)

#### Model: Association Rules

#### Model: AI